In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Data/Listening_Data/songs_listened_to_with_all_info_unique.csv')
df = df.drop('Unnamed: 0.1', axis=1)
df = df.drop('Unnamed: 0', axis = 1)
df

,endTime,artistName,trackName,albumName,msPlayed,month,year,Season,trackUri,danceability,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1/20/2022 4:50,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,60,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4
1,1/20/2022 4:53,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,213851,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4
2,1/20/2022 19:48,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,0,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4
3,1/20/2022 21:59,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,213851,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4
4,1/21/2022 2:55,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,213851,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14121,1/18/2023 0:00,Shordie Shordie,Seattle,Memory Lane,153546,1,2023,1,spotify:track:7CBmiFdp5vj83mstdl3Qmu,0.577,...,0.4230,0.677,113.020,audio_features,7CBmiFdp5vj83mstdl3Qmu,spotify:track:7CBmiFdp5vj83mstdl3Qmu,https://api.spotify.com/v1/tracks/7CBmiFdp5vj8...,https://api.spotify.com/v1/audio-analysis/7CBm...,153547,5
14122,1/19/2023 18:52,Shordie Shordie,Seattle,Memory Lane,153546,1,2023,1,spotify:track:7CBmiFdp5vj83mstdl3Qmu,0.577,...,0.4230,0.677,113.020,audio_features,7CBmiFdp5vj83mstdl3Qmu,spotify:track:7CBmiFdp5vj83mstdl3Qmu,https://api.spotify.com/v1/tracks/7CBmiFdp5vj8...,https://api.spotify.com/v1/audio-analysis/7CBm...,153547,5
14123,1/20/2023 20:02,Shordie Shordie,Seattle,Memory Lane,153546,1,2023,1,spotify:track:7CBmiFdp5vj83mstdl3Qmu,0.577,...,0.4230,0.677,113.020,audio_features,7CBmiFdp5vj83mstdl3Qmu,spotify:track:7CBmiFdp5vj83mstdl3Qmu,https://api.spotify.com/v1/tracks/7CBmiFdp5vj8...,https://api.spotify.com/v1/audio-analysis/7CBm...,153547,5
14124,1/20/2023 4:08,Kodak Black,Skrt,Skrt,224864,1,2023,1,spotify:track:5v7kaZNsnyByrSJOfO8gKq,0.901,...,0.1220,0.199,111.065,audio_features,5v7kaZNsnyByrSJOfO8gKq,spotify:track:5v7kaZNsnyByrSJOfO8gKq,https://api.spotify.com/v1/tracks/5v7kaZNsnyBy...,https://api.spotify.com/v1/audio-analysis/5v7k...,224865,4


In [3]:
one_hot_df = pd.concat([df.drop('month', axis=1), pd.get_dummies(df['Season'])], axis=1)
one_hot_df.head(2)

,endTime,artistName,trackName,albumName,msPlayed,year,Season,trackUri,danceability,energy,...,id,uri,track_href,analysis_url,duration_ms,time_signature,1,2,3,4
0,1/20/2022 4:50,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,60,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,0.693,...,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4,1,0,0,0
1,1/20/2022 4:53,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,213851,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,0.693,...,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4,1,0,0,0


In [4]:
one_hot_df.columns

Index([         'endTime',       'artistName',        'trackName',
              'albumName',         'msPlayed',             'year',
                 'Season',         'trackUri',     'danceability',
                 'energy',              'key',         'loudness',
                   'mode',      'speechiness',     'acousticness',
       'instrumentalness',         'liveness',          'valence',
                  'tempo',             'type',               'id',
                    'uri',       'track_href',     'analysis_url',
            'duration_ms',   'time_signature',                  1,
                        2,                  3,                  4],
      dtype='object')

In [5]:
df['Count'] = 1
count_df = df.groupby('trackName', as_index = False).sum('Count')
count_df[['trackName', 'Count']]

,trackName,Count
0,#1 Freak (feat. Ty Dolla $ign),9
1,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,1
2,1 0 0 . m i l ‘ (with Bas),1
3,1-800-273-8255,2
4,10 F*CKS (with Mansa),7
...,...,...
1845,upsides,11
1846,viola,1
1847,way back,7
1848,when the water meets the mountains (acoustic v...,2


In [31]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets

features = one_hot_df[['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 1, 2, 3, 4]]
features.columns = features.columns.astype(str)
labels = pd.merge(one_hot_df, count_df, on='trackName')['Count']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.1, random_state = 42)

In [ ]:
#gridsearchcv, randomsearchcv

In [32]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, min_samples_leaf=10)
# Train the model on training data
rf.fit(features, labels)


RandomForestRegressor(n_estimators=1000, random_state=42)

In [38]:
#XG Boost

from xgboost import XGBRFClassifier
model = XGBRFClassifier(n_estimators=100, subsample=0.9, colsample_bynode=0.35, learning_rate=0.1)
model.fit(train_features, train_labels)

/Users/jamesyang/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

/Users/jamesyang/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



[09:16:22] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=0.35, colsample_bytree=1,
                enable_categorical=False, gamma=0, gpu_id=-1,
                importance_type=None, interaction_constraints='',
                learning_rate=0.1, max_delta_step=0, max_depth=6,
                min_child_weight=1, missing=nan, monotone_constraints='()',
                n_estimators=100, n_jobs=8, num_parallel_tree=100,
                objective='multi:softprob', predictor='auto', random_state=0,
                reg_alpha=0, scale_pos_weight=None, subsample=0.9,
                tree_method='exact', validate_parameters=1, verbosity=None)

In [40]:
y_pred_test = model.predict(test_features)
test = test_labels.values.reshape(1,len(test_labels))
test = test[0]

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
mse = mean_squared_error(y_pred_test, test)
rmse = mse**.5
print(mse)
print(rmse)
mean_absolute_percentage_error(test,y_pred_test)

161.37721160651097
12.703433063802516


1.6301477578016854

In [8]:
rf.fit(train_features, train_labels)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [9]:
y_pred_test = rf.predict(test_features)
test = test_labels.values.reshape(1,len(test_labels))
test = test[0]

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
mse = mean_squared_error(y_pred_test, test)
rmse = mse**.5
print(mse)
print(rmse)

15.183955995569441
3.896659594520599


In [11]:
mean_absolute_percentage_error(test,y_pred_test)

0.5871631564580455

In [12]:
rf_train = rf.fit(train_features, train_labels)

In [13]:
max(test)

186

In [14]:
song_stats = pd.read_csv('Data/Playlist_Data/playlist_stats.csv').drop('Unnamed: 0', axis = 1)
song_stats.head(5)

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,1,2,3,4,trackName
0,0.436,0.655,1,-8.370,0.0583,0.4990,0.000008,0.688,0.412,121.002,0,0,0,1,Good Days
1,0.603,0.784,6,-4.023,0.0620,0.4460,0.000008,0.119,0.769,172.041,0,0,0,1,Dark Red
2,0.913,0.603,8,-4.892,0.2260,0.0272,0.000503,0.167,0.497,123.061,0,0,0,1,Lost
3,0.586,0.893,7,-8.772,0.4390,0.8270,0.642000,0.212,0.650,140.023,0,0,0,1,Funny Thing
4,0.535,0.388,7,-9.805,0.1550,0.5070,0.000000,0.110,0.412,106.703,0,0,0,1,I Hate U


In [15]:
import numpy as np
# Use the forest's predict method on the test data

predictions = rf.predict(song_stats.drop('trackName', axis = 1))
# test_labels = test_labels['Count']

In [16]:
# LightGBM 


In [17]:
#Get the output from the predictions

prediction_df = pd.DataFrame()
prediction_df['Count'] = predictions
prediction_df['trackName'] = song_stats['trackName']
prediction_df['color'] = 'pred'
prediction_df.sort_values('Count', ascending=False)
# prediction_df.to_csv('Data/Playlist_Data/Season4.csv')

,Count,trackName,color
13,78.000,Strawberry Skies,pred
3,13.440,Funny Thing,pred
0,9.938,Good Days,pred
12,8.903,Super Rich Kids,pred
1,8.582,Dark Red,pred
10,7.183,"Dreams, Fairytales, Fantasies (feat. Brent Fai...",pred
14,6.744,DEVASTATED,pred
9,6.658,White Ferrari,pred
11,6.636,Novacane,pred
6,6.111,Love Is Only a Feeling,pred


In [18]:
original_labels = pd.merge(one_hot_df, count_df, on='trackName')[['Count', 'trackName']]
original_labels['color'] = 'original'
original_labels.head(5)

,Count,trackName,color
0,73,Make the Most (feat. H.E.R.),original
1,73,Make the Most (feat. H.E.R.),original
2,73,Make the Most (feat. H.E.R.),original
3,73,Make the Most (feat. H.E.R.),original
4,73,Make the Most (feat. H.E.R.),original


In [19]:
total_labels = original_labels.append(prediction_df)
total_labels.head(5)

/var/folders/h4/6452mjq15618sb_49rqm1mkw0000gn/T/ipykernel_3922/1640145732.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_labels = original_labels.append(prediction_df)


,Count,trackName,color
0,73.0,Make the Most (feat. H.E.R.),original
1,73.0,Make the Most (feat. H.E.R.),original
2,73.0,Make the Most (feat. H.E.R.),original
3,73.0,Make the Most (feat. H.E.R.),original
4,73.0,Make the Most (feat. H.E.R.),original


In [20]:
import plotly.express as px
hovertemplate = "Track: %{customdata[0]} <br> Count: %{customdata[1]} </br>"
fig = px.scatter(total_labels.sort_values('Count'), x = 'Count', y = 'Count', color='color', custom_data=['trackName', 'Count'])
fig.update_traces(hovertemplate=hovertemplate)
fig.show()

In [21]:
labels = labels.append(prediction_df['Count'])
labels

/var/folders/h4/6452mjq15618sb_49rqm1mkw0000gn/T/ipykernel_3922/1529402440.py:1: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



0     73.000
1     73.000
2     73.000
3     73.000
4     73.000
       ...  
10     7.183
11     6.636
12     8.903
13    78.000
14     6.744
Name: Count, Length: 14141, dtype: float64

In [22]:
labels = labels.drop_duplicates()
labels['normalized_count'] = labels /labels.abs().max()
labels

0                                                                73.0
73                                                                2.0
75                                                               51.0
126                                                             146.0
272                                                              95.0
                                          ...                        
10                                                              7.183
11                                                              6.636
12                                                              8.903
14                                                              6.744
normalized_count    0      0.392473
73     0.010753
75     0.27419...
Name: Count, Length: 94, dtype: object